# Báo cáo Đồ án Song song hóa và tối ưu hóa một ứng dụng
- Thông tin nhóm
  - Lê Thanh Hiếu - 1712434
  - Nguyễn Văn Tuấn - 1512638
  - Phan Hữu Tú - 1712861

## 1. Mô tả ứng dụng
- **Ứng dụng**: **Thay đổi kích thước ảnh bằng Seam Carving**
    - **Input**: một tấm ảnh (RGB)
    - **Output**: một tấm ảnh mới được thay đổi kích thước (không chỉ cho chiều rộng mà còn cho chiều cao, không chỉ thu nhỏ mà còn phóng to) từ tấm ảnh gốc mà không làm biến dạng các đối tượng quan trọng.
- **Ý nghĩa thực tế khi sử dụng?**
    - Khi muốn có các phiên bản với các kích thước khác nhau của cùng một tấm ảnh để hiển thị trên các thiết bị khác nhau (máy tính, điện thoại, ...)
   - Một ví dụ thực tế khi tạo thumbnail (ảnh đại diện) cho 1 video chúng ta sẽ dựa vào nhưng yếu tố khác để resize ảnh về đúng khung hình ta mong muốn như màn hình điện thoại 16:9 , 3:4

- **Ứng dụng này có cần tăng tốc không?**  
    - Vì seam carving rất chậm khi cài đặt tuần tự và có tiềm năng song song hóa.<br/>
    ![](readmeimages/seam_carving.png) <br/>
> Nguồn ảnh: Slides Lập trình song song HK1


## 2. Cài đặt tuần tự


### 2.1. Thiết kế
- **Ý tưởng**: 
    - Một seam: một tập các pixel, mỗi dòng một pixel,pixel của dòng r & dòng r+1 được kết nối với nhau
    - Việc thay đổi kích thước ảnh được thực hiện bằng cách loại bỏ đi các điểm ảnh ít quan trọng và giữ lại các điểm ảnh quan trọng.


#### 2.1.1. Các bước triển khai thuật toán

![](readmeimages/workflow.png)

##### Thuật toán xóa seam theo cột
  - Bước 1: Chuyển ảnh từ RGB sang Grayscale
  - Bước 2: Tính bảng Energy của ảnh
  - Bước 3: Tìm bảng chi phí nhỏ nhất
  - Bước 4: Tìm seam có chi phí nhỏ nhất
  - Bước 5: Xóa seam nhỏ nhất tìm được ở bước 4
  - Bước 6: Lặp lại bước 5 cho tới khi xóa  đủ số seams cần xóa


##### Thuật toán xóa seam theo hàng
  - Bước 1: Xoay ảnh một góc 90 độ theo chiều kim đồng hồ
  - Bước 2: Lặp lại thuật toán xóa seam theo cột cho đến khi xóa đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

##### Thuật toán chèn seam theo cột
  - Bước 1: Thực hiện thuật toán xóa seam theo cột trên một ảnh được sao chép từ ảnh gốc và lưu lại danh sách các điểm ảnh đã xóa theo đúng thứ tự đã xóa
  - Bước 2: Chèn các seams vào ảnh gốc theo đúng thứ tự tọa độ đã lưu. Với giá trị của pixel được chèn được lấy bằng trung bình của 2 pixels kế bên.

##### Thuật toán chèn seam theo hàng
  - Bước 1: Xoay ảnh 90 độ theo chiều kim đồng hồ
  - Bước 2: Thực hiện thuật toán chèn seam theo cột cho đến khi đủ số seams cần xóa
  - Bước 3: Xoay ảnh lại một góc 90 độ ngược chiều kim đồng hồ

#### 2.1.2. Chi tiết thuật toán

##### Xây dựng bảng Energy
- Chuyển ảnh từ rgb sang graysale
- Dùng edge detetion
    - Phát hiện ảnh theo chiều x (1): thực hiện convolution giữa ảnh grayscale với bộ lọc x-Sobel
    - Phát hiện ảnh theo chiều y (2): thực hiện convolution giữa ảnh grayscale với bộ lọc y-Sobel
- Độ quan trọng của mỗi pixel = abs(kết quả tương ứng của (1)) + abs(kết quả tương ứng của (2))

##### Xây dựng bảng chi phí nhỏ nhất
- Sử dụng Dynamic Programming để xây dựng bảng chi phí nhỏ nhất
    - Ở hàng trên cùng ta có tổng chi phí giữ nguyên. Với các hàng còn lại, chi phí của mỗi pixel bằng engergy của nó cộng với chi phí nhỏ nhất của 3 pixel liền kề phía trên. <br/>
    ![](readmeimages/algo1.png)<br/>
    > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project

    - Lặp lại quá trình cho đến hàng cuối cùng ta được bảng chi phí nhỏ nhất. <br/>
    ![](readmeimages/algo2.png)
    > Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project


##### Xác định seam nhỏ nhất
- Từ bảng chi phí nhỏ nhất trên ta xác định được vị trí có tổng chi phí nhỏ nhất ở hàng cuối cùng và quay lui để xác định seam nhỏ nhất<br/>
![](readmeimages/algo3.png)
> Nguồn ảnh: https://github.com/kalpeshdusane/Seam-Carving-B.E.-Project

### 2.2. Đánh giá

#### 2.2.1 Đánh giá về kết quả

| Số seams |         forward_energy          | get_minimum_seam |                  Xóa theo cột                   |                Xóa theo hàng                 |                  Chèn theo cột                  |                Chèn theo hàng                 |
| :------: | :--------------------------: | :--------------: | :---------------------------------------------: | :------------------------------------------: | :---------------------------------------------: | :-------------------------------------------: |
|    1     | ![](images/ouput_forward_energy.jpg) |        -         |                        -                        |                      -                       |                        -                        |                       -                       |
|    50    |              -               |        -         | ![](images/output_remove50seams_by_column.jpg)  | ![](images/output_remove50seams_by_row.jpg)  | ![](images/output_insert50seams_by_column.jpg)  | ![](images/output_insert50seams_by_rows.jpg)  |
|   100    |              -               |        -         | ![](images/output_remove100seams_by_column.jpg) | ![](images/output_remove100seams_by_row.jpg) | ![](images/output_remove100seams_by_column.jpg) | ![](images/output_insert100seams_by_rows.jpg) |
|   200    |              -               |        -         | ![](images/output_remove200seams_by_column.jpg) | ![](images/output_remove200seams_by_row.jpg) | ![](images/output_insert200seams_by_column.jpg) | ![](images/output_insert200seams_by_rows.jpg) |
|   500    |              -               |        -         | ![](images/output_remove500seams_by_column.jpg) | ![](images/output_remove500seams_by_row.jpg)  | ![](images/output_insert500seams_by_column.jpg) | ![](images/output_insert500seams_by_rows.jpg) |
|   900    |              -               |        -         | ![](images/output_remove900seams_by_column.jpg) | ![](images/output_remove900seams_by_row.jpg) | ![](images/output_insert900seams_by_column.jpg) | ![](images/output_insert900seams_by_rows.jpg) |

#### 2.2.2. Đánh giá về thời gian

| Số seams | calc_energy | get_minimum_seam | Xóa theo cột | Xóa theo hàng | Chèn theo cột | Chèn theo hàng |
| :------: | :---------: | :--------------: | :----------: | :-----------: | :-----------: | :------------: |
|    1     |   1.137s    |      1.847s      |      -       |       -       |       -       |       -        |
|    50    |      -      |        -         |   13.965s    |    14.156s    |    14.102s    |    14.897s     |
|   100    |      -      |        -         |   24.388s    |    24.803s    |    28.791s    |    27.947s     |
|   200    |      -      |        -         |   51.509s    |    44.815s    |    52.849s    |    50.820s     |
|   500    |      -      |        -         |   105.254s   |    91.250s    |   118.467s    |    109.687s    |
|   900    |      -      |        -         |   157.822s   |   121.856s    |   185.654s    |    153.134s    |

Tổng thời gian: 1409.179s

## 3. Cài đặt song song

### 3.1. Phân tích

__Đo thời gian của các phần của phiên bản tuần tự để quyết định thành phần cần song song hóa__

| Hàm              | Thời gian chạy |
| ---------------- | -------------- |
| rgb2gray         | 0.0348s        |
| forward_energy   | 0.1124s        |
| get_minimum_seam | 0.0560s        |
| remove_seam      | 0.0383s        |
| insert_seam      | 0.0235s        |

Nhận thấy rằng các hàm trong quá trình thực hiện thuật toán seam carving có thời gian chạy tương đối lớn và đều có tiềm năng song song hóa nên nhóm tiến hành song song hóa tất cả các bước trên.

### 3.2. Thiết kế

#### 3.2.1. Chuyển ảnh RGB sang ảnh xám trên hàm Kernel 

- Mỗi thread sẽ chịu trách nhiệm tính kết quả grayscale cho một pixel.

#### 3.2.2. Tính bảng năng lượng trên hàm Kernel 

#### 3.2.3. Tìm seam có chi phí nhỏ nhất trên hàm Kernel 

Để tìm seam có chi phí nhỏ nhất ta cần tìm bảng chi phí nhỏ nhất.  
- Vì chi phí nhỏ nhất của mỗi pixel đươc tính bằng cách cộng thêm chi phí nhỏ nhất của 1 trong 3 pixel kề trên nên ta có thể song song hóa việc tính toán theo từng dòng.
- Khi gọi hàm kernel này, threads sẽ được tạo cho mỗi column, bảng chi phí nhỏ nhất được tạo theo từng dòng, với mỗi thread sẽ tính chi phí cho một pixel trên dòng đó.  
- Vì mỗi dòng của bảng chi phí nhỏ nhất sẽ phụ thuộc vào dòng phía trên, những threads sẽ phải synchronize sau mỗi dòng.  

Sau khi hoàn tất quá trình xây dựng bảng chi phí nhỏ nhất, ta xác định được pixel có chi phí nhỏ nhất ở dòng cuối cùng của bảng. Bắt đầu từ pixel đó, ta sẽ quay lui để tìm seam cần xóa.

#### 3.2.4. Xóa seam trên hàm Kernel 

#### 3.2.5. Chèn seam trên hàm Kernel 

## 4. Tối ưu hóa cài đặt song song

## 5. Nhìn lại quá trình làm đồ án

## 6. Tài liệu tham khảo
- https://vnoi.info/wiki/cs/imageprocessing/Seam-Carving.md
- https://en.wikipedia.org/wiki/Seam_carving#cite_note-ACM1276390-1


- Song song
  - https://shwestrick.github.io/2020/07/29/seam-carve.html